<a href="https://colab.research.google.com/github/deepak2233/Fake-News-Classifier-Using-Bidirectional-LSTM/blob/main/Fake_News_Classifier_Using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [6]:
df=pd.read_csv('/content/train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
### Drop Nan Values
df=df.dropna()

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:

## Get the Dependent features
y=df['label']

In [12]:

y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [17]:
### Vocabulary size
voc_size=5000

## Onehot Representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [24]:

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3853, 3071, 4734, 4922, 3067, 3855, 4104, 131, 4736, 1705],
 [1030, 3613, 4315, 3745, 3092, 1984, 2339],
 [3322, 2369, 3320, 1566],
 [114, 276, 924, 276, 913, 2184],
 [1536, 3092, 3537, 442, 1983, 4108, 3092, 4763, 4044, 3639],
 [3108,
  1804,
  2387,
  1492,
  1387,
  1487,
  2204,
  2428,
  3884,
  2622,
  965,
  4031,
  2008,
  2794,
  2339],
 [2315, 3836, 1584, 1660, 3755, 818, 1405, 2923, 1561, 4519, 4106],
 [4855, 1332, 4477, 2510, 4618, 4454, 1487, 1512, 1561, 4519, 4106],
 [2807, 218, 3835, 540, 3047, 2459, 2833, 1068, 1487, 4200],
 [1372, 3746, 455, 4149, 2921, 816, 1537, 112],
 [2, 3890, 3140, 2828, 3528, 3505, 340, 2440, 3227, 1108, 1918],
 [276, 314, 3067, 2459, 1487, 4618],
 [4890, 1935, 716, 2782, 3540, 1869, 3932, 4239, 2891],
 [4133, 121, 2308, 1694, 2287, 2982, 3147, 1561, 4519, 4106],
 [2629, 1983, 1040, 2019, 2040, 1561, 4519, 4106],
 [1484, 1240, 4807, 3207, 1584, 1703, 1379, 3454, 3930, 594],
 [1432, 3593, 3613],
 [4361, 3479, 3888, 3879, 1487, 2248, 2716, 2339],

## Embedding Representation

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  131 4736 1705]
 [   0    0    0 ... 3092 1984 2339]
 [   0    0    0 ... 2369 3320 1566]
 ...
 [   0    0    0 ... 1561 4519 4106]
 [   0    0    0 ... 1602  240 2887]
 [   0    0    0 ...  466 2136  962]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3853,
       3071, 4734, 4922, 3067, 3855, 4104,  131, 4736, 1705], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:

## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:

len(embedded_docs),y.shape

(18285, (18285,))

In [30]:

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [31]:

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [32]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [33]:

### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 49ms/step - loss: 0.4419 - accuracy: 0.7630 - val_loss: 0.2003 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 8s 43ms/step - loss: 0.1306 - accuracy: 0.9503 - val_loss: 0.2365 - val_accuracy: 0.9089
Epoch 3/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0851 - accuracy: 0.9704 - val_loss: 0.2254 - val_accuracy: 0.9117
Epoch 4/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0644 - accuracy: 0.9779 - val_loss: 0.3155 - val_accuracy: 0.9054
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0464 - accuracy: 0.9857 - val_loss: 0.3936 - val_accuracy: 0.9029
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0265 - accuracy: 0.9920 - val_loss: 0.3941 - val_accuracy: 0.9062
Epoch 7/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0172 - accuracy: 0.9953 - val_loss: 0.4682 - val_accuracy: 0.902

## Performance Metrics And Accuracy


In [34]:
y_pred1=model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:
from sklearn.metrics import confusion_matrix

In [36]:
confusion_matrix(y_test,y_pred1)

array([[3081,  338],
       [ 246, 2370]])

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9032311516155758

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

